In [531]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

In [532]:
frases = ["Cacá já mandou 3 gol contra, chega uma hora que não é mais azar, é cabacisse mesmo",
          "Caca tem que começar a ser punido por isso plmrddeus como é que pode",
          "Eu nunca vi um zagueiro fazer tanto gol contra bicho",
          "PÉSSIMO... EU TO CANSADO",
          "GRANDE HUGO KKKKKKKKKKKKKKKK",
          "Já pode fechar as portas porque esse time está fadado ao fracasso.",
          "TAVA DEMORANDO MSM KKKKKKKKKKKKLKKKKKKKKKK INÚTEIS",
          "o que ta acontecendo com meu time mds",
          "nao conseguimos passar do meio campo contra o CRICIUMA",
          "ainda pensei em apostar no timão"]

In [533]:
vectorizer = CountVectorizer()

In [534]:
bow = vectorizer.fit_transform(frases)
vocabulario = vectorizer.get_feature_names_out()

In [535]:
vetores_bag = bow.toarray()

Aplicação da rede neural c/ backpropagation

In [536]:
entradas = np.array(vetores_bag)
saidas = np.array([[1, 0, 0],
                   [1, 0, 0],
                   [0, 1, 0],
                   [1, 0, 0],
                   [0, 1, 0],
                   [1, 0, 0],
                   [1, 0, 0],
                   [0, 1, 0],
                   [0, 1, 0],
                   [0, 1, 0]])

In [537]:
entradas_size = entradas.shape[1]
camada_oculta_size = 37
saidas_sizes = saidas.shape[1]
taxa_aprendizado = 0.01
epocas = 1000000

In [538]:
w_entrada_oculta = np.random.uniform(-1, 1, (entradas_size, camada_oculta_size))
b_camada_oculta = np.random.uniform(-1, 1, camada_oculta_size)
w_oculta_saida = np.random.uniform(-1, 1, (camada_oculta_size, saidas_sizes))
b_saida = np.random.uniform(-1, 1, saidas_sizes)

In [539]:
def relu(z):
    return np.maximum(0, z)

In [540]:
def relu_derivative(z):
    return np.where(z > 0, 1, 0)

In [541]:
def softmax(z):
    if z.ndim > 1:
        e_z = np.exp(z - np.max(z, axis=1, keepdims=True))
        return e_z / np.sum(e_z, axis=1, keepdims=True)
    else:
        e_z = np.exp(z - np.max(z))
        return e_z / np.sum(e_z)

In [542]:
for epoca in range(epocas):
    for i in range(len(entradas)):
        
        z_oculta = np.dot(entradas[i], w_entrada_oculta) + b_camada_oculta
        a_oculta = relu(z_oculta)
        z_saida = np.dot(a_oculta, w_oculta_saida) + b_saida
        a_saida = softmax(z_saida)
        
        erro_saida = saidas[i] - a_saida
        
        delta_saida = erro_saida
        gradiente_w_oculta_saida = np.outer(a_oculta, delta_saida)
        gradiente_b_saida = delta_saida
        delta_oculta = np.dot(delta_saida, w_oculta_saida.T) * relu_derivative(z_oculta)
        gradiente_w_entrada_oculta = np.outer(entradas[i], delta_oculta)
        gradiente_b_oculta = delta_oculta
        
        w_oculta_saida += taxa_aprendizado * gradiente_w_oculta_saida
        b_saida += taxa_aprendizado * gradiente_b_saida
        w_entrada_oculta += taxa_aprendizado * gradiente_w_entrada_oculta
        b_camada_oculta += taxa_aprendizado * gradiente_b_oculta
        
    if epoca % 100000 == 0:
        erro_medio = np.mean(np.square(erro_saida))
        print(f'Época {epoca}, Erro médio: {erro_medio}')

Época 0, Erro médio: 0.25295471693877275
Época 100000, Erro médio: 1.3348990974681163e-11
Época 200000, Erro médio: 2.929556759151672e-12
Época 300000, Erro médio: 1.2111770153261613e-12
Época 400000, Erro médio: 6.476840869565373e-13
Época 500000, Erro médio: 3.979856786376197e-13
Época 600000, Erro médio: 2.6797813362592613e-13
Época 700000, Erro médio: 1.9200404613408258e-13
Época 800000, Erro médio: 1.439168600652104e-13
Época 900000, Erro médio: 1.1163714436136855e-13


In [543]:
def interpretar_saida(resultado_bin):
    if resultado_bin[0] == 1:
        return 'Negativo'
    elif resultado_bin[1] == 1:
        return 'Neutro'
    elif resultado_bin[2] == 1:
        return 'Positivo'
    else:
        return 'Indefinido'

In [544]:
for j in range(len(entradas)):
    z_oculta = np.dot(entradas[j], w_entrada_oculta) + b_camada_oculta
    a_oculta = relu(z_oculta)
    z_saida = np.dot(a_oculta, w_oculta_saida) + b_saida
    a_saida = softmax(z_saida.reshape(1, -1))
    
    resultado_bin = np.round(a_saida).astype(int)[0]
    
    interpretacao = interpretar_saida(resultado_bin)
    
    print(f'Tweet: "{frases[j]}"')
    print(f'Saída Esperada: {saidas[j]}, Resultado: {resultado_bin} ({interpretacao})')

Tweet: "Cacá já mandou 3 gol contra, chega uma hora que não é mais azar, é cabacisse mesmo"
Saída Esperada: [1 0 0], Resultado: [1 0 0] (Negativo)
---
Tweet: "Caca tem que começar a ser punido por isso plmrddeus como é que pode"
Saída Esperada: [1 0 0], Resultado: [1 0 0] (Negativo)
---
Tweet: "Eu nunca vi um zagueiro fazer tanto gol contra bicho"
Saída Esperada: [0 1 0], Resultado: [0 1 0] (Neutro)
---
Tweet: "PÉSSIMO... EU TO CANSADO"
Saída Esperada: [1 0 0], Resultado: [1 0 0] (Negativo)
---
Tweet: "GRANDE HUGO KKKKKKKKKKKKKKKK"
Saída Esperada: [0 1 0], Resultado: [0 1 0] (Neutro)
---
Tweet: "Já pode fechar as portas porque esse time está fadado ao fracasso."
Saída Esperada: [1 0 0], Resultado: [1 0 0] (Negativo)
---
Tweet: "TAVA DEMORANDO MSM KKKKKKKKKKKKLKKKKKKKKKK INÚTEIS"
Saída Esperada: [1 0 0], Resultado: [1 0 0] (Negativo)
---
Tweet: "o que ta acontecendo com meu time mds"
Saída Esperada: [0 1 0], Resultado: [0 1 0] (Neutro)
---
Tweet: "nao conseguimos passar do meio campo co

In [545]:
print('--- Pesos finais ---')
print('Pesos da camada de entrada para oculta:')
print(w_entrada_oculta)
print('Bias da camada oculta:')
print(b_camada_oculta)
print('Pesos da camada oculta para saída:')
print(w_oculta_saida)
print('Bias da camada de saída:')
print(b_saida)

--- Pesos finais ---
Pesos da camada de entrada para oculta:
[[-0.56600252 -0.8002499   0.93609546 ... -0.90612169  0.47138263
   0.62404762]
 [ 0.5635168  -0.55056342  0.80765878 ...  0.42305976  0.58774569
   0.38790611]
 [ 0.90802102 -0.31317088  0.896894   ... -0.22107097 -0.93505161
  -0.21484643]
 ...
 [ 0.83555601  0.31920993 -0.87696247 ...  0.21014488 -0.4994414
  -0.56379548]
 [ 0.21599659  0.4973405   0.31735712 ...  0.58249034 -0.69921366
  -0.25401987]
 [ 0.46176711  1.113803   -0.74321926 ...  0.39942624 -0.44673374
   0.8975912 ]]
Bias da camada oculta:
[ 0.37988973  0.39257089 -0.6676847  -0.15672302  0.53987266 -0.64244187
 -0.35118281  0.54760029  0.92112694 -0.73352419  0.62828825  0.81887103
 -0.61737423  0.72795468 -0.54322223  0.2090337   0.47923908  0.73952004
  0.20268566 -0.6499475  -0.10219194 -0.94790439  0.63432109 -0.67296745
 -0.91453269  0.74527566  0.28223566  0.21906427  0.90917757 -0.56204779
 -0.08068433 -0.60062867  0.47318478 -0.62316064 -0.51746892

In [561]:
nova_frase = ["Time inútil estou cansado"]

vectorizer = CountVectorizer(vocabulary=vocabulario)

nova_entrada = vectorizer.transform(nova_frase).toarray()


z_oculta = np.dot(nova_entrada, w_entrada_oculta) + b_camada_oculta
a_oculta = relu(z_oculta)
z_saida = np.dot(a_oculta, w_oculta_saida) + b_saida
a_saida = softmax(z_saida)

resultado_bin = np.round(a_saida).astype(int)[0]

interpretacao = interpretar_saida(resultado_bin)

print(f'Novo Tweet: "{nova_frase[0]}"')
print(f'Resultado: {resultado_bin} ({interpretacao})')


Novo Tweet: "Time inútil estou cansado"
Resultado: [1 0 0] (Negativo)
